In [89]:
import requests
import boto3
import json
import os
import pandas as pd
from dotenv import load_dotenv

In [90]:
# Search for .env in the parent directory
load_dotenv("../.env")

True

In [91]:
# pydantic_models.py
from pydantic import BaseModel, Field, field_validator, ConfigDict
from datetime import date, datetime

class DailyStockData(BaseModel):
    # Modern Pydantic V2 Configuration
    model_config = ConfigDict(populate_by_name=True)
    
    symbol: str
    date: date
    open_price: float = Field(alias="1. open")
    high_price: float = Field(alias="2. high")
    low_price: float = Field(alias="3. low")
    close_price: float = Field(alias="4. close")
    volume: int = Field(alias="5. volume")

    @field_validator("open_price", "high_price", "low_price", "close_price")
    @classmethod  # V2 field_validators must be class methods
    def price_must_be_positive(cls, v: float) -> float:
        if v <= 0:
            raise ValueError("Stock prices must be greater than zero")
        return v

In [92]:
def fetch_daily_data(BASE_URL: str, stock_ticker: str, API_KEY: str) -> dict:
    """Fetches raw JSON from the API."""
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": stock_ticker,
        "outputsize": "compact", #only pull past 90 days of data
        "apikey": API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

In [93]:
BASE_URL="https://www.alphavantage.co/query"
ALPHA_VANTAGE_API_KEY= os.getenv("ALPHA_VANTAGE_API_KEY")

stock_tickers = ["AAPL", "MSFT"]

In [94]:
raw_data = fetch_daily_data(BASE_URL=BASE_URL, stock_ticker= stock_tickers[0], API_KEY=ALPHA_VANTAGE_API_KEY)
time_series = raw_data.get("Time Series (Daily)", {})

In [95]:
# 1. Get the sorted dates (newest first)
all_dates = sorted(time_series.keys(), reverse=True)

In [96]:
# 2. Slice the first 7 dates
latest_7_dates = all_dates[:7]
latest_7_dates

['2026-01-15',
 '2026-01-14',
 '2026-01-13',
 '2026-01-12',
 '2026-01-09',
 '2026-01-08',
 '2026-01-07']

In [97]:
validated_records = []

for date_str in latest_7_dates:
    metrics = time_series[date_str]
    # Validate each day using your Pydantic model
    record = DailyStockData(
        symbol=stock_tickers[0],
        date=date_str,
        **metrics
    )
    validated_records.append(record)

In [98]:
validated_records

[DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 15), open_price=260.65, high_price=261.04, low_price=257.05, close_price=258.21, volume=39388564),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 14), open_price=259.49, high_price=261.82, low_price=256.71, close_price=259.96, volume=40019421),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 13), open_price=258.72, high_price=261.81, low_price=258.39, close_price=261.05, volume=45730847),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 12), open_price=259.16, high_price=261.3, low_price=256.8, close_price=260.25, volume=45263767),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 9), open_price=259.075, high_price=260.21, low_price=256.22, close_price=259.37, volume=39996967),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 1, 8), open_price=257.02, high_price=259.29, low_price=255.7, close_price=259.04, volume=50419337),
 DailyStockData(symbol='AAPL', date=datetime.date(2026, 

In [99]:
df = pd.DataFrame([r.model_dump() for r in validated_records])
df['created_at'] = datetime.now()

In [100]:
df

,symbol,date,open_price,high_price,low_price,close_price,volume,created_at
0,AAPL,2026-01-15,260.650,261.04,257.05,258.21,39388564,2026-01-15 21:36:55.219257
1,AAPL,2026-01-14,259.490,261.82,256.71,259.96,40019421,2026-01-15 21:36:55.219257
2,AAPL,2026-01-13,258.720,261.81,258.39,261.05,45730847,2026-01-15 21:36:55.219257
3,AAPL,2026-01-12,259.160,261.30,256.80,260.25,45263767,2026-01-15 21:36:55.219257
4,AAPL,2026-01-09,259.075,260.21,256.22,259.37,39996967,2026-01-15 21:36:55.219257
5,AAPL,2026-01-08,257.020,259.29,255.70,259.04,50419337,2026-01-15 21:36:55.219257
6,AAPL,2026-01-07,263.200,263.68,259.81,260.33,48309804,2026-01-15 21:36:55.219257
